In [37]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [39]:
pretrain_features = {
    "learning_rate": 0.1,
    "batch_size": 256,
    "epochs": 1000,
    "eval_size": 4*256,
    "momentum": 0.005,
    "weight_decay": 0.0001,
}

In [40]:
def load_data():
    """
    This function loads the data from the csv files and returns it as numpy arrays.

    input: None
    
    output: x_pretrain: np.ndarray, the features of the pretraining set
            y_pretrain: np.ndarray, the labels of the pretraining set
            x_train: np.ndarray, the features of the training set
            y_train: np.ndarray, the labels of the training set
            x_test: np.ndarray, the features of the test set
    """
    x_pretrain = pd.read_csv("./pretrain_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_pretrain = pd.read_csv("./pretrain_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_train = pd.read_csv("./train_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_train = pd.read_csv("./train_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_test = pd.read_csv("./test_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1)
    
#     scaler = StandardScaler()
#     x_pretrain = scaler.fit_transform(x_pretrain)
#     x_train = scaler.transform(x_train)
#     x_test_transed = scaler.transform(x_test)
#     x_test[x_test.columns] = x_test_transed

    return x_pretrain, y_pretrain, x_train, y_train, x_test

In [41]:
x_pretrain, y_pretrain, x_train, y_train, x_test = load_data()

In [42]:
class AE(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(1000, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.6),
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.6),
            nn.Linear(64, 10),
            nn.BatchNorm1d(10),
            nn.LeakyReLU(0.01)
            )

        self.decoder = nn.Sequential(
            nn.Linear(10, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.6),
            nn.Linear(64, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.6),
            nn.Linear(256, 1000),
            nn.BatchNorm1d(1000),
            nn.LeakyReLU(0.01)
            )
            
        for m in self.modules():
            if isinstance(m, nn.Linear):    
                nn.init.xavier_uniform_(m.weight)
        
        
    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def encode(self, x):
        return self.encoder(x)

In [43]:
eval_size = 1000
batch_size = 256
learning_rate = 0.01
ae_model = AE()
ae_model.train()
ae_model.to(device)

def train_autoencoder():
    x_tr, x_val, y_tr, y_val = train_test_split(x_pretrain, y_pretrain, test_size=eval_size, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    optimizer = torch.optim.Adam(ae_model.parameters(), lr=learning_rate)
    # optimizer = torch.optim.SGD(ae_model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = 1000
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, _] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            predictions = ae_model(x)
            loss = criterion(predictions, x)
            loss.backward()
            optimizer.step()
            loss_tr += loss.item() * len(x)
        loss_tr /= len(train_loader.dataset)
        for [x, _] in val_loader:
            x = x.to(device)
            predictions = ae_model(x)
            loss = criterion(predictions, x)
            loss_val += loss.item() * len(x)
        loss_val /= len(val_loader.dataset)
        scheduler.step(loss_val)
        
        # if(epoch % 10 == 0):
        print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-6):
            print(f"Early stop at epoch {epoch+1}")
            break

train_autoencoder()

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1: train loss: 0.04505546429935767, val loss: 0.03530123350024223
Epoch 2: train loss: 0.03440218642962222, val loss: 0.033902372807264325
Epoch 3: train loss: 0.03327326464896299, val loss: 0.03287018379569054
Epoch 4: train loss: 0.03244164434622745, val loss: 0.032234469830989834
Epoch 5: train loss: 0.031941096130682495, val loss: 0.03181082752346993
Epoch 6: train loss: 0.03155850038844712, val loss: 0.03137721699476242
Epoch 7: train loss: 0.031229984399615503, val loss: 0.031096578821539878
Epoch 8: train loss: 0.03095118919197394, val loss: 0.030863363310694693
Epoch 9: train loss: 0.030726539033712173, val loss: 0.030610124066472053
Epoch 10: train loss: 0.030495185870297103, val loss: 0.03046785219013691
Epoch 11: train loss: 0.03030294287356795, val loss: 0.030390532821416856
Epoch 12: train loss: 0.03012161080508816, val loss: 0.03020090335607529
Epoch 13: train loss: 0.02995653474087618, val loss: 0.030020905196666718
Epoch 14: train loss: 0.029797275884723178, val l

In [44]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, HuberRegressor, ElasticNet
def get_regression_model(X, y):
    """
    This function returns the regression model used in the pipeline.

    input: None

    output: model: sklearn compatible model, the regression model
    """
    # model = HuberRegressor(alpha=0.001, max_iter=10000)
    model = ElasticNet(alpha=0.01, l1_ratio=0.05, max_iter=10000)
    model.fit(X, y)
    return model

In [45]:
result_file = "results-ae-lr.csv"

In [46]:
featured_x_train = ae_model.encode(torch.tensor(x_train, dtype=torch.float).to(device)).detach().cpu().numpy()
scaler = StandardScaler()
featured_x_train = scaler.fit_transform(featured_x_train)
print(featured_x_train.shape)

y_pred = np.zeros(x_test.shape[0])
featured_x_test = ae_model.encode(torch.tensor(x_test.to_numpy(), dtype=torch.float).to(device))
featured_x_test = scaler.transform(featured_x_test.detach().cpu().numpy())
# featured_x_test = torch.tensor(featured_x_test, dtype=torch.float).to(device)
# y_pred = one_model(featured_x_test).squeeze(-1).detach().cpu().numpy()
lr = get_regression_model(featured_x_train, y_train)
print(lr.score(featured_x_train, y_train))
y_pred = lr.predict(featured_x_test)

assert y_pred.shape == (x_test.shape[0],)
y_pred = pd.DataFrame({"y": y_pred}, index=x_test.index)
y_pred.to_csv(result_file, index_label="Id")
print(f"Predictions saved to {result_file}, all done!")

(100, 10)
0.4548999383664605
Predictions saved to results-ae-lr.csv, all done!
